In [1]:
from sklearn import datasets
import math
import numpy as np

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = datasets.load_iris().data

In [3]:
def euclidean_distance(data1, data2):
    sum = 0
    if (len(data1) == len(data2)):
        for x1, x2 in zip(data1, data2):
            sum += (x1 - x2)**2
        dist = math.sqrt(sum)
        return dist
    else:
        raise Exception('Length doesn\'t match')
        
def manhattan_distance(data1, data2):
    sum = 0
    if (len(data1) == len(data2)):
        for x1, x2 in zip(data1, data2):
            sum += abs(x1 - x2)
        return sum
    else:
        raise Exception('Length doesn\'t match')
        
def get_distance(data1, data2, metrics):
    if (metrics == 'euclidean'):
        dist = euclidean_distance(data1, data2)
    elif (metrics == 'manhattan'):
        dist = manhattan_distance(data1, data2)
    else:
        raise Exception('Metrics not defined')
    return dist

In [4]:
def calculate_distance_matrix(data, metrics):
    dist_matrix = []
    for idx1, data1 in enumerate(data):
        curr_dist_matrix = []
        for idx2, data2 in enumerate(data):
            if (idx1 > idx2):
                curr_dist_matrix.append(dist_matrix[idx2][idx1])
            else:
                dist = get_distance(data1, data2, metrics)
                curr_dist_matrix.append(dist)
        dist_matrix.append(curr_dist_matrix)
    return dist_matrix

In [5]:
metrics_used = 'euclidean'
data_used = data[:50]
dist_matrix = calculate_distance_matrix(data_used, metrics_used)

In [6]:
def complete_linkage(cluster1, cluster2, dist_matrix):
    max_dist = 0
    for v1 in cluster1:
        for v2 in cluster2:
            if (max_dist < dist_matrix[v1][v2]):
                max_dist = dist_matrix[v1][v2]
    return max_dist

def single_linkage(cluster1, cluster2, dist_matrix):
    min_dist = None
    for v1 in cluster1:
        for v2 in cluster2:
            if (min_dist is None) or (min_dist < dist_matrix[v1][v2]):
                min_dist = dist_matrix[v1][v2]
    return min_dist

def average_linkage(cluster1, cluster2, dist_matrix):
    sum_dist = 0
    count_dist = 0
    for v1 in cluster1:
        for v2 in cluster2:
            sum_dist += dist_matrix[v1][v2]
            count_dist += 1
    return float(sum_dist)/float(count_dist)

def group_average_linkage(cluster1, cluster2, data, distance):
    data1 = [data[i] for i in cluster1]
    data2 = [data[i] for i in cluster2]
    
    avg1 = np.mean(data1, axis = 0)
    avg2 = np.mean(data2, axis = 0)
    
    return get_distance(avg1, avg2, distance)

In [22]:
clusters = [[i] for i, c in enumerate(data_used)]
n_clusters = 4
linkage = 'average'

while(len(clusters) > n_clusters):
    min_dist = None
    merge_pair = (0, 0)
    for idx1, c1 in enumerate(clusters):
        for idx2, c2 in enumerate(clusters[(idx1 + 1) :]):
            if (linkage == 'single'):
                dist = single_linkage(c1, c2, dist_matrix)
            elif (linkage == 'complete'):
                dist = complete_linkage(c1, c2, dist_matrix)
            elif (linkage == 'average'):
                dist = average_linkage(c1, c2, dist_matrix)
            elif (linkage == 'average_group'):
                dist = group_average_linkage(c1, c2, data_used, metrics_used)
            else:
                raise Exception('Linkage not defined')
            if (min_dist == None) or (dist < min_dist):
                min_dist = dist
                merge_pair = (idx1, idx1 + 1 + idx2)
    
    result_cluster = []
    for idx, c in enumerate(clusters):
        if idx not in merge_pair:
            result_cluster.append(c)
    
    result_cluster.append(clusters[merge_pair[0]] + clusters[merge_pair[1]])
    
    clusters = result_cluster

print(clusters)
result_per_item = np.zeros(len(data_used), dtype= np.int8)
for idx, clust in enumerate(clusters):
    result_per_item[clust] = idx

result_per_item

[[41], [36, 20, 31, 43, 23, 26, 35, 4, 40, 0, 17, 27, 28, 49, 7, 39, 44, 5, 18, 16, 10, 48, 46, 19, 21], [14, 15, 32, 33], [22, 13, 42, 8, 38, 11, 24, 29, 30, 12, 1, 45, 25, 37, 9, 34, 6, 2, 3, 47]]


array([1, 3, 3, 3, 1, 1, 3, 1, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1,
       3, 1, 3, 3, 1, 1, 1, 3, 3, 1, 2, 2, 3, 1, 1, 3, 3, 1, 1, 0, 3, 1,
       1, 3, 1, 3, 1, 1], dtype=int8)

In [18]:
from sklearn.cluster import AgglomerativeClustering

cls = AgglomerativeClustering(n_clusters= 4, linkage='average', affinity='manhattan')
cls.fit_predict(data[:50])

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 1, 3, 0, 1,
       1, 0, 1, 0, 1, 1])